#### This Code caluclates the Elevation_Difference Uncertainty based on the Standard Error Approach from Gardelle et al. 2013
### For further reading, please refer Gardelle et al. 2013 (doi:10.5194/tc-7-1263-2013) or Goerlich et al. 2017 (doi:10.3390/rs9030275)

In [1]:
#importing the libraries required for the evaluation
import numpy as np
import rasterio
from pygeotools.lib import malib, warplib, iolib, geolib
import numpy.ma as ma
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

#### Creating two seperate dataframes for glacier and bedrock areas 
##### Calculating stats for both the cases
##### Joining both dataframes based on the index at Rounded Elevation value

In [2]:
#dataframe (df) for glacier pixels
fn1='Elevation.tif' #Elevation_dataset, it is assumed that both the elevation and dh rasters are clipped to glacier extents
fn2='dh_glaciers.tif' #Elevation_difference dataset
ds_list=warplib.memwarp_multi_fn([fn1,fn2],res='max',extent='intersection',t_srs='first', r='cubic')
A=iolib.ds_getma(ds_list[0])
dh=iolib.ds_getma(ds_list[1])
E=ma.asarray(A,dtype=np.float32)
dh_glacier=ma.compressed(dh)
DEM_glacier=ma.compressed(E)
raw={'Elevation':DEM_glacier,
    'dh':dh_glacier}
df=pd.DataFrame(raw, columns=['Elevation','dh'])
#dataframe (df1) for bedrock (ice-free stable_Areas)
fn3='Stable_dh_Elevation_total.tif' #Elevation_dataset, it is assumed that both the elevation and dh rasters are clipped to glacier extents
fn4='Stable_dh.tif' #Elevation_difference dataset
ds_list_1=warplib.memwarp_multi_fn([fn3,fn4],res='max',extent='intersection',t_srs='first', r='cubic')
B=iolib.ds_getma(ds_list_1[0])
dh1=iolib.ds_getma(ds_list_1[1])
E1=ma.asarray(B,dtype=np.float32)
dh_stable=ma.compressed(dh1)
DEM_stable=ma.compressed(E1)
raw_stable={'Elevation':DEM_stable,
    'dh':dh_stable}
df2=pd.DataFrame(raw_stable, columns=['Elevation','dh'])
#Rounding Glacier Elevation to next nearest multiple of 50
df['Elevation_rounded']=(df['Elevation']+49)//50*50
#Calulating_stats_for_Glacier_Pixels
grouped=df.groupby('Elevation_rounded')
df1=grouped['dh'].agg([np.count_nonzero,np.sum, np.mean, np.std])
#Rounding Bedrock Elevation to next nearest multiple of 50
df2['Elevation_rounded']=(df2['Elevation']+49)//50*50
#Calulating_stats_for_Bedrock_Pixels
grouped_stable=df2.groupby('Elevation_rounded')
df3=grouped_stable['dh'].agg([np.count_nonzero,np.sum, np.mean, np.std])
#Joining the stats dataframes (df1*Glacier* and df3*Stable*) into a common"df4"
df4=df3.join(df1,how='left', lsuffix="Stable", rsuffix="Glacier", sort=False)
df4


Warping all inputs to the following:
Resolution: 30.0
Extent: [594711.267, 3697416.167, 631071.267, 3753786.167]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: cubic

1 of 2: Elevation.tif
2 of 2: dh_glaciers.tif

Warping all inputs to the following:
Resolution: 30.0
Extent: [594726.267, 3697431.167, 631056.267, 3753771.167]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: cubic

1 of 2: Stable_dh_Elevation_total.tif
2 of 2: Stable_dh.tif


,count_nonzeroStable,sumStable,meanStable,stdStable,count_nonzeroGlacier,sumGlacier,meanGlacier,stdGlacier
Elevation_rounded,,,,,,,,
3250.0,15.0,-242.808655,-16.187244,7.889551,NaN,NaN,NaN,NaN
3300.0,40.0,-30.612206,-0.765305,10.865774,NaN,NaN,NaN,NaN
3350.0,291.0,-63.505035,-0.218230,6.089076,NaN,NaN,NaN,NaN
3400.0,1592.0,-1631.704102,-1.024940,4.720934,NaN,NaN,NaN,NaN
3450.0,1420.0,-2557.948730,-1.801372,7.861134,NaN,NaN,NaN,NaN
3500.0,1302.0,-166.882980,-0.128174,8.342449,NaN,NaN,NaN,NaN
3550.0,2184.0,-1269.671997,-0.581352,6.566604,83.0,15.213154,0.183291,6.283049
3600.0,2361.0,37.315208,0.015805,9.464114,459.0,-4188.076172,-9.124349,9.402762
3650.0,2662.0,2650.398682,0.995642,10.070865,1655.0,-29696.236328,-17.943346,15.289424


In [4]:
#Effective_No_of_Pixels "Equation 5 in Goerlich et al. 2017"
df4['Effective_stable_pixels']=(df4['count_nonzeroStable']*30)/(2*191)
distance_of_auto_correlation=191 #Calculated from semivariogram for the bedrock dh values, working on the script of semivariogram
df4['SE']=(df4['stdStable'])/(np.sqrt(df4['Effective_stable_pixels']))#Standard Error for each interval "Equation 4 in Goerlich et al. 2017"

In [5]:
#Weighing the Standard Error in each interval with glacier hypsometry of the corresponding interval
df4['SE*Count_Glacier']=df4['SE']*df4['count_nonzeroGlacier']
SUM_SE_Count=df4['SE*Count_Glacier'].sum()
Sum_Count=df4['count_nonzeroGlacier'].sum()
Elevation_difference_error=(SUM_SE_Count)/(Sum_Count)
print'Error_in_Elevation_difference_estimate is %s m' %Elevation_difference_error

Error_in_Elevation_difference_estimate is 1.7716 m
